In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import os
from os.path import join
import re
import time
import argparse
import pickle

# in-house tools
import gdreg

# autoreload
%load_ext autoreload
%autoreload 2

In [7]:
# Required
JOB = "compute_ld"
PGEN_FILE = "/n/groups/price/martin/data_GDREG/toy_10K/chr@_v1.SPB.hg19.toy_10K"
PREFIX_OUT = "/n/groups/price/martin/data_GDREG/toy_10K/res_gdreg/test"
SNP_RANGE = 'c1_s20_e1701_rall'

# Filler
LD_FILE = None
SCORE_FILE = None
SUMSTATS_FILE = None
ANNOT_FILE = None
MEMORY = 1024
RANDOM_SEED = 0
FLAG_FULL_LD = False
FLAG_CROSS_TERM = False

In [8]:
sys_start_time = time.time()

###########################################################################################
######                                    Parse Options                              ######
###########################################################################################

# JOB = args.job
# PGEN_FILE = args.pgen_file
# LD_FILE = args.ld_file
# SCORE_FILE = args.score_file
# SUMSTATS_FILE = args.sumstats_file
# ANNOT_FILE = args.annot_file
# PREFIX_OUT = args.prefix_out
# SNP_RANGE = args.snp_range
# MEMORY = args.memory
# RANDOM_SEED = args.random_seed
# FLAG_FULL_LD = args.flag_full_ld
# FLAG_CROSS_TERM = args.flag_cross_term

# Parse and check arguments
LEGAL_JOB_LIST = ["get_snp_block", "compute_ld", "compute_score", "regress"]
err_msg = "# run_gdreg: --job=%s not supported" % JOB
assert JOB in LEGAL_JOB_LIST, err_msg

if JOB in ["compute_score"]:
    assert LD_FILE is not None, "--ld_file required for --job=%s" % JOB
if JOB in ["regress"]:
    assert SCORE_FILE is not None, "--score_file required for --job=%s" % JOB
if JOB in ["regress"]:
    assert SUMSTATS_FILE is not None, "--sumstats_file required for --job=%s" % JOB
if JOB in ["compute_score", "regress"]:
    assert ANNOT_FILE is not None, "--annot_file required for --job=%s" % JOB
if JOB in ["compute_ld"]:
    assert SNP_RANGE is not None, "--snp_range required for --job=%s" % JOB
    DIC_RANGE = gdreg.util.parse_snp_range(SNP_RANGE)

# Print input options
header = gdreg.util.get_cli_head()
header += "Call: run_gdreg.py \\\n"
header += "--job %s\\\n" % JOB
header += "--pgen_file %s\\\n" % PGEN_FILE
header += "--ld_file %s\\\n" % LD_FILE
header += "--score_file %s\\\n" % SCORE_FILE
header += "--sumstats_file %s\\\n" % SUMSTATS_FILE
header += "--annot_file %s\\\n" % ANNOT_FILE
header += "--prefix_out %s\\\n" % PREFIX_OUT
header += "--snp_range %s\\\n" % SNP_RANGE
header += "--memory %d\\\n" % MEMORY
header += "--random_seed %d\\\n" % RANDOM_SEED
header += "--flag_full_ld %s\\\n" % FLAG_FULL_LD
header += "--flag_cross_term %s\n" % FLAG_CROSS_TERM
print(header)

******************************************************************************
* Gene-level directional effect regression (GDREG)
* Version 0.0.2
* Martin Jinye Zhang
* HSPH / Broad Institute
* MIT License
******************************************************************************
Call: run_gdreg.py \
--job compute_ld\
--pgen_file /n/groups/price/martin/data_GDREG/toy_10K/chr@_v1.SPB.hg19.toy_10K\
--ld_file None\
--score_file None\
--sumstats_file None\
--annot_file None\
--prefix_out /n/groups/price/martin/data_GDREG/toy_10K/res_gdreg/test\
--snp_range c1_s20_e1701_rall\
--memory 1024\
--random_seed 0\
--flag_full_ld False\
--flag_cross_term False



In [4]:
###########################################################################################
######                                   Data Loading                                ######
###########################################################################################
# Load --pgen_file
if JOB in ["compute_ld", "compute_score", "regress"]:
    print("# Loading --pgen_file")
    dic_data = {}
    if "@" not in PGEN_FILE:
        temp_dic = gdreg.util.read_pgen(PGEN_FILE)
        dic_data[temp_dic["pvar"]["CHR"][0]] = temp_dic.copy()
    else:
        for CHR in range(1, 23):
            if os.path.exists(PGEN_FILE.replace("@", "%s" % CHR) + ".pgen"):
                dic_data[CHR] = gdreg.util.read_pgen(
                    PGEN_FILE.replace("@", "%s" % CHR)
                )

    for CHR in dic_data:
        n_sample = dic_data[CHR]["psam"].shape[0]
        n_snp = dic_data[CHR]["pvar"].shape[0]
        mat_X = gdreg.util.read_geno(
            dic_data[CHR]["pgen"], 0, 50, n_sample=None, n_snp=None
        )
        sparsity = (mat_X != 0).mean()
        print(
            "    CHR%2d: %d samples, %d SNPs, %0.1f%% non-zeros for first 50 SNPs"
            % (CHR, n_sample, n_snp, sparsity * 100)
        )
    print("    " + gdreg.util.get_sys_info(sys_start_time))

# Load --ld_file
if JOB in ["compute_score", "regress"]:
    print("# Loading --ld_file")
    dic_ld = {}
    for CHR in dic_data:
        err_msg = "--ld_file missing for CHR%d" % CHR
        assert os.path.exists(LD_FILE.replace("@", "%s" % CHR)), err_msg
        if LD_FILE.endswith(".full_ld.npy"):
            dic_ld[CHR] = np.load(LD_FILE.replace("@", "%s" % CHR))
        elif LD_FILE.endswith(".ld.npz"):
            dic_ld[CHR] = sp.sparse.load_npz(LD_FILE.replace("@", "%s" % CHR))
        err_msg = "CHR%2d n_snp=%d, mismatch with --pgen_file" % (
            CHR,
            dic_ld[CHR].shape[0],
        )
        assert dic_ld[CHR].shape[0] == dic_data[CHR]["pvar"].shape[0], err_msg
    print("    LD info loaded, matching --pgen_file")
    print("    " + gdreg.util.get_sys_info(sys_start_time))

# Load --sumstats_file
if JOB in ["regress"]:
    print("# Loading --sumstats_file")
    df_sumstats = pd.read_csv(SUMSTATS_FILE, sep="\t", index_col=None)
    print("    .sumstats.gz loaded, %d SNPs" % df_sumstats.shape[0])
    print("    " + gdreg.util.get_sys_info(sys_start_time))

# Load --annot_file
if JOB in ["compute_score", "regress"]:
    print("# Loading --annot_file")
    df_annot = None
    pannot_list = []
    pannot_hr_list = []
    for annot_file in ANNOT_FILE.split(","):
        err_msg = "--annot_file missing : '%s'" % annot_file
        assert os.path.exists(annot_file), err_msg
        temp_df = gdreg.util.read_annot(annot_file)

        if annot_file.endswith(".annot.gz"):
            temp_df.index = temp_df["SNP"]
            if df_annot is None:
                df_annot = temp_df.copy()
            else:
                col_list = [x for x in temp_df if x.startswith("AN:")]
                df_annot = df_annot.join(temp_df[col_list])
        if annot_file.endswith(".pannot.gz"):
            pannot_list.append(temp_df.copy())
        if annot_file.endswith(".pannot_hr.gz"):
            pannot_hr_list.append(temp_df.copy())
    AN_list = [x for x in df_annot if x.startswith("AN:")]
    print(
        "    .annot.gz (%d SNPs and %d annots): %s"
        % (df_annot.shape[0], len(AN_list), ",".join(AN_list))
    )
    temp_list = ["%s (%d SNPs)" % (x.columns[-1], x.shape[0]) for x in pannot_list]
    print(
        "    .pannot.gz (%d pannots): %s" % (len(pannot_list), ",".join(temp_list)),
    )
    temp_list = [
        "%s (%d pairs)" % (x.columns[-1], x.shape[0]) for x in pannot_hr_list
    ]
    print(
        "    .pannot_hr.gz (%d pannots): %s"
        % (len(pannot_hr_list), ",".join(temp_list)),
    )
    print("    " + gdreg.util.get_sys_info(sys_start_time))

# Loading --pgen_file
    CHR 1: 10000 samples, 4232 SNPs, 26.7% non-zeros for first 50 SNPs
    CHR 2: 10000 samples, 4056 SNPs, 43.5% non-zeros for first 50 SNPs
    CHR 3: 10000 samples, 4067 SNPs, 33.9% non-zeros for first 50 SNPs
    CHR 4: 10000 samples, 4027 SNPs, 24.2% non-zeros for first 50 SNPs
    CHR 5: 10000 samples, 4106 SNPs, 36.7% non-zeros for first 50 SNPs
    CHR 6: 10000 samples, 4154 SNPs, 39.6% non-zeros for first 50 SNPs
    CHR 7: 10000 samples, 4071 SNPs, 41.8% non-zeros for first 50 SNPs
    CHR 8: 10000 samples, 3891 SNPs, 33.3% non-zeros for first 50 SNPs
    CHR 9: 10000 samples, 4149 SNPs, 40.1% non-zeros for first 50 SNPs
    CHR10: 10000 samples, 4129 SNPs, 40.3% non-zeros for first 50 SNPs
    sys_time=16.0s, sys_mem=0.09GB


In [13]:
if JOB == "compute_ld":
    print("# Running --job compute_ld")
    if FLAG_FULL_LD:
        CHR, CHR_REF = DIC_RANGE["chr"], DIC_RANGE["chr_ref"][0]
        pos_tar = [CHR, 0, dic_data[CHR]["pvar"].shape[0]]
        pos_ref = [CHR_REF, 0, dic_data[CHR_REF]["pvar"].shape[0]]
        mat_ld = gdreg.score.compute_ld(
            dic_data, pos_tar, pos_ref, verbose=True, memory=MEMORY
        )
#         np.save(PREFIX_OUT + ".c%s_r%s_fullld" % (CHR, CHR_REF), mat_ld)
    else:
        CHR, START, END = DIC_RANGE["chr"], DIC_RANGE["start"], DIC_RANGE["end"]
        n_snp_tar, n_snp = (END - START), dic_data[CHR]["pvar"].shape[0]
        v_bp = dic_data[CHR]["pvar"]["BP"].values

        block_size = 1000
        n_block = np.ceil(n_snp_tar / block_size).astype(int)
        mat_ld_list = []
        for i_block in range(n_block):
            print(
                "block %d/%d %s"
                % (i_block, n_block, gdreg.util.get_sys_info(sys_start_time))
            )
            ind_s = START + i_block * block_size
            ind_e = min(START + (i_block + 1) * block_size, END)
            ind_s_ref = np.searchsorted(v_bp, v_bp[ind_s] - 5.01e6, side="left")
            ind_s_ref = max(0, ind_s_ref - 1)
            ind_e_ref = np.searchsorted(
                v_bp, v_bp[ind_e - 1] + 5.01e6, side="right"
            )
            ind_e_ref = min(n_snp, ind_e_ref + 1)

            pos_tar = [CHR, ind_s, ind_e]
            pos_ref = [CHR, ind_s_ref, ind_e_ref]
            mat_ld = gdreg.score.compute_ld(
                dic_data,
                pos_tar,
                pos_ref,
                verbose=True,
                memory=MEMORY,
            )
            temp_mat = np.zeros([n_snp, ind_e - ind_s], dtype=np.float32)
            temp_mat[ind_s_ref:ind_e_ref, :] = mat_ld
            mat_ld_list.append(sp.sparse.csc_matrix(temp_mat))

        mat_ld = sp.sparse.hstack(mat_ld_list, format="csc")
#         sp.sparse.save_npz(PREFIX_OUT + ".%s_ld" % SNP_RANGE, mat_ld)
    print("    " + gdreg.util.get_sys_info(sys_start_time))

# Running --job compute_ld
block 0/2 sys_time=1669.6s, sys_mem=0.14GB
# Call: gdreg.score.compute_ld
    n_snp_tar=1000 (CHR1), n_snp_ref=1445 (CHR1), n_sample=10000
    memory=1024MB
    block_size_tar=1000, n_block_tar=1
    block_size_ref=1445, n_block_ref=1
    block_size_sample=16383, n_block_sample=1
    Completed, time=4.0s
block 1/2 sys_time=1673.7s, sys_mem=0.15GB
# Call: gdreg.score.compute_ld
    n_snp_tar=681 (CHR1), n_snp_ref=1595 (CHR1), n_sample=10000
    memory=1024MB
    block_size_tar=681, n_block_tar=1
    block_size_ref=1595, n_block_ref=1
    block_size_sample=16383, n_block_sample=1
    Completed, time=3.0s
    sys_time=1676.8s, sys_mem=0.14GB


In [25]:
mat_ld,dic_range = gdreg.util.read_ld(
    "/n/groups/price/martin/data_GDREG/toy_10K/gdreg_file_ld/toy_10K.c1_s0_e4232_ld.npz"
)
mat_ld_ref,dic_range_ref = gdreg.util.read_ld(
    "/n/groups/price/martin/data_GDREG/toy_10K/gdreg_file_fullld/toy_10K.c1_r1_fullld.npy"
)
print('dif=%0.4g' % np.absolute(mat_ld[mat_ld!=0] - mat_ld_ref[mat_ld.toarray()!=0]).sum())

dif=0


0.0

In [20]:
mat_ld_ref.shape

(4232, 4232)

### Tests for gdreg.score.compute_ld

In [14]:
# Correctness
for pos_tar,pos_ref in [
    [[1, 0, 100], [1, 5, 98]],
    [[1, 109, 220], [1, 5, 98]],
    [[1, 109, 220], [2, 5, 98]],
    [[1, 0, 1400], [10, 5, 98]],
    [[2, 0, 1300], [10, 1279, 1280]]
]:
    print('pos_tar=%s, pos_ref=%s' % (
        ','.join(['%d'%x for x in pos_tar]), 
        ','.join(['%d'%x for x in pos_ref])
    ))
    mat_X = gdreg.util.read_geno(dic_data[pos_tar[0]]["pgen"], pos_tar[1], pos_tar[2])
    mat_X = mat_X.T.astype(np.float32)
    mat_X[mat_X == -9] = 0
    v_maf = mat_X.mean(axis=0) * 0.5
    mat_X = (mat_X - 2 * v_maf) / np.sqrt(2 * v_maf * (1 - v_maf))

    mat_Y = gdreg.util.read_geno(dic_data[pos_ref[0]]["pgen"], pos_ref[1], pos_ref[2])
    mat_Y = mat_Y.T.astype(np.float32)
    mat_Y[mat_Y == -9] = 0
    v_maf = mat_Y.mean(axis=0) * 0.5
    mat_Y = (mat_Y - 2 * v_maf) / np.sqrt(2 * v_maf * (1 - v_maf))

    mat_ld_gold = mat_Y.T.dot(mat_X) / 10000
    
    mat_ld = gdreg.score.compute_ld(dic_data, pos_tar, pos_ref, verbose=False, memory=128)

    print('    abs_dif=%0.3g' % (
        np.absolute(mat_ld_gold - mat_ld).mean()
    ))

pos_tar=1,0,100, pos_ref=1,5,98
    abs_dif=2.85e-07
pos_tar=1,109,220, pos_ref=1,5,98
    abs_dif=6.84e-08
pos_tar=1,109,220, pos_ref=2,5,98
    abs_dif=5.21e-08
pos_tar=1,0,1400, pos_ref=10,5,98
    abs_dif=5.21e-08
pos_tar=2,0,1300, pos_ref=10,1279,1280
    abs_dif=5.65e-08
